In [ ]:
!pip install faker
from faker import Faker
import csv
import json

registros = []

# Função para inserir novos registros
def criar_registro(tipo, valor, taxa_juros=None, tempo_aplicacao=None):
    if tipo not in ["despesa", "receita", "investimento"]:
        print("Tipo inválido. Escolha entre despesa, receita ou investimento.")
        return
    try:
        valor = float(valor)
    except:
        print("Valor inválido. Digite um número.")
        return
    registro = {}
    registro["tipo"] = tipo
    registro["valor"] = valor
    from datetime import date
    hoje = date.today()
    registro["data"] = hoje
    if tipo == "despesa":
        registro["valor"] = -valor
    # Investimento: calcular o montante, taxa de juros e tempo de aplicação...
    if tipo == "investimento":
        if taxa_juros is None or tempo_aplicacao is None:
            print("Taxa de juros e tempo de aplicação são obrigatórios para investimentos.")
            return
        i = taxa_juros / 100 # Converte a taxa de juros para decimal
        M = valor * (1 + i) ** tempo_aplicacao # Aplica a fórmula do montante
        M = round(M, 2) # Arredondar o montante para duas casas decimais
        registro["montante"] = M
        registro["taxa_juros"] = taxa_juros
        registro["tempo_aplicacao"] = tempo_aplicacao
    registros.append(registro)
    print("Registro criado com sucesso.")

faker = Faker()
Faker.seed(29) #Garantir que os registros gerados sejam sempre iguais.

# Função para criar e exibir 10 registros aleatórios iniciais
def criar_registros_faker():
    for _ in range(10):
        tipo = faker.random_element(elements=("despesa", "receita", "investimento"))
        valor = faker.random_number(digits=4)
        if tipo == "investimento":
            taxa_juros = faker.random_number(digits=1)
            tempo_aplicacao = faker.random_number(digits=1)
            criar_registro(tipo, valor, taxa_juros, tempo_aplicacao)
        else:
            criar_registro(tipo, valor)

criar_registros_faker()
for i, registro in enumerate(registros):
    print(f"Registro {i + 1}:")
    print(f"Tipo: {registro['tipo']}")
    print(f"Valor: {registro['valor']}")
    print(f"Data: {registro['data']}")
    if registro["tipo"] == "investimento":
        print(f"Montante: {registro['montante']}")
        print(f"Taxa de Juros: {registro['taxa_juros']}")
        print(f"Tempo de Aplicação: {registro['tempo_aplicacao']} anos")
    print()

# Função para ler registros por data, tipo ou valor
def ler_registros(filtro=None, valor=None):
    if filtro not in [None, "data", "tipo", "valor"]:
        print("Filtro inválido. Escolha entre data, tipo ou valor.")
        return
    filtrados = []
    if filtro == None:  # Se o filtro for None, copia todos os registros
        filtrados = registros.copy()

    else: # Se o filtro for data, valor ou tipo, puxa os registros que correspondem ao valor
        for registro in registros:
            if str(registro.get(filtro)) == str(valor): # Convertemos os valores em strings antes de comparar para evitar erros
                filtrados.append(registro)

    if len(filtrados) == 0:
        print("Nenhum registro encontrado.")
        return

    print("Registros encontrados:")
    for i, registro in enumerate(filtrados):
        print(f"Registro {i + 1}:")
        print(f"Tipo: {registro['tipo']}")
        print(f"Valor: {registro['valor']}")
        print(f"Data: {registro['data']}")
        if registro["tipo"] == "investimento":
            print(f"Montante: {registro['montante']}")
        print()

# Função para atualizar os registros
def atualizar_registro(indice, tipo=None, valor=None):
    if indice < 0 or indice >= len(registros):
        print("Índice inválido. Escolha um índice entre 0 e", len(registros) - 1)
        return

    registro = registros[indice]
    print("\nInformações do Registro:")
    print(f"Índice: {indice}")
    print(f"Tipo: {registro['tipo']}")
    print(f"Valor: {registro['valor']}")
    print(f"Data: {registro['data']}")
    if registro["tipo"] == "investimento":
        print(f"Montante: {registro['montante']}")
        print(f"Taxa de Juros: {registro['taxa_juros']}")
        print(f"Tempo de Aplicação: {registro['tempo_aplicacao']} anos")

    print("\nDigite as modificações (deixe em branco para manter o valor atual):")
    novo_tipo = input(f"Novo Tipo ({registro['tipo']}): ") or registro['tipo']
    novo_valor = input(f"Novo Valor ({registro['valor']}): ") or registro['valor']

    if novo_tipo not in [None, "despesa", "receita", "investimento"]:
        print("Tipo inválido. Escolha entre despesa, receita ou investimento.")
        return

    try:
        novo_valor = float(novo_valor)
    except ValueError:
        print("Valor inválido. Digite um número.")
        return

    registro["tipo"] = novo_tipo
    registro["valor"] = novo_valor

    from datetime import date
    hoje = date.today()
    registro["data"] = hoje

    if novo_tipo == "despesa":
        registro["valor"] = -novo_valor

    # Se o tipo for investimento, calcula o montante
    if novo_tipo == "investimento":
        i = float(input("Digite a taxa de juros anual (em porcentagem): "))
        t = float(input("Digite o tempo de aplicação (em anos): "))
        i = i / 100
        M = novo_valor * (1 + i) ** t
        M = round(M, 2)
        registro["montante"] = M
    print("Registro atualizado com sucesso.")

# Função para deletar registros
def deletar_registro(indice):
  if indice < 0 or indice >= len(registros):
    print("Índice inválido. Escolha um índice entre 0 e", len(registros)-1)
    return
  registros.pop(indice)
  print("Registro deletado com sucesso.")

def agrupar_registros(filtro):
    if filtro not in ["mes", "tipo"]:
        print("Filtro inválido. Escolha entre mês ou tipo.")
        return

    grupos = {}

    for registro in registros:
        if filtro == "mes": # Se o filtro for mês, obter o mês da data do registro
            mes = registro["data"].month
            if mes not in grupos: # Se o mês não estiver no dicionário, criar uma chave com o mês e um valor inicial zero
                grupos[mes] = 0
            grupos[mes] += registro["valor"]
        elif filtro == "tipo": # Se o filtro for tipo, obter o tipo do registro
            tipo = registro["tipo"]
            if tipo not in grupos:# Se o tipo não estiver no dicionário, criar uma chave com o tipo e um valor inicial zero
                grupos[tipo] = 0
            grupos[tipo] += registro["valor"]

    # Exibir os resultados
    print("Resultados do agrupamento:")
    for chave, valor in grupos.items():
        print(f"{chave}: {valor}")

    # Retornar o dicionário com as somas
    return grupos

# Função para exportar o relatório em CSV ou JSON
def exportar_relatorio(formato):
    if formato not in ['csv', 'json']:
        print("Formato inválido. Escolha entre 'csv' ou 'json'.")
        return
    nome_arquivo = f"relatorio.{formato}"

    if formato == 'csv':
        with open(nome_arquivo, mode='w', newline='') as file:
            writer = csv.writer(file)
            # Escrever o cabeçalho
            writer.writerow(["Tipo", "Valor", "Data", "Montante"])
            # Escrever os registros
            for registro in registros:
                writer.writerow([registro.get("tipo", ""), registro.get("valor", ""),
                                 registro.get("data", ""), registro.get("montante", "")])

    elif formato == 'json':
        with open(nome_arquivo, mode='w') as file:
            json.dump(registros, file, indent=2)

    print(f"Relatório no formato {formato} exportado com sucesso .")

# Função para exibir menu do usuário
def exibir_menu():
    print("\nEscolha uma opção:")
    print("1. Inserir registro")
    print("2. Ler registros")
    print("3. Atualizar registro")
    print("4. Deletar registro")
    print("5. Agrupar registros")
    print("6. Exportar relatório")
    print("7. Sair")

while True:
    exibir_menu()
    opcao = input("Digite o número da opção desejada: ")

    if opcao == '1':

      print("Escolha o tipo do registro que será inserido:")
      print("1. Receita")
      print("2. Despesa")
      print("3. Investimento")

      escolha_tipo = input("Digite o número correspondente ao tipo desejado: ")

      if escolha_tipo == '1':
        tipo = 'receita'
      elif escolha_tipo == '2':
        tipo = 'despesa'
      elif escolha_tipo == '3':
        tipo = 'investimento'
      else:
        print("Opção inválida. Tente novamente.")
        continue

      valor = input("Digite o valor: ")

      if tipo == 'investimento':
        taxa_juros = float(input("Digite a taxa de juros anual (em porcentagem): "))
        tempo_aplicacao = float(input("Digite o tempo de aplicação (em anos): "))
        criar_registro(tipo, valor, taxa_juros, tempo_aplicacao)
      else:
        criar_registro(tipo, valor)

    elif opcao == '2':
        filtro = input("Digite o filtro (data, tipo, valor): ")
        valor = input("Digite o valor do filtro: ")
        ler_registros(filtro, valor)

    elif opcao == '3':
        indice = int(input("Digite o índice do registro a ser atualizado: "))
        atualizar_registro(indice)

    elif opcao == '4':
        indice = int(input("Digite o índice do registro a ser deletado: "))
        deletar_registro(indice)

    elif opcao == '5':
        filtro = input("Digite o filtro para agrupar (mes, tipo): ")
        agrupar_registros(filtro)

    elif opcao == '6':
        formato = input("Digite o formato para exportar (csv, json): ")
        exportar_relatorio(formato)

    elif opcao == '7':
        break

    else:
        print("Opção inválida. Tente novamente.")


Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro criado com sucesso.
Registro 1:
Tipo: investimento
Valor: 1247.0
Data: 2023-11-22
Montante: 1934.51
Taxa de Juros: 5
Tempo de Aplicação: 9 anos

Registro 2:
Tipo: investimento
Valor: 4728.0
Data: 2023-11-22
Montante: 5119.75
Taxa de Juros: 1
Tempo de Aplicação: 8 anos

Registro 3:
Tipo: receita
Valor: 6437.0
Data: 2023-11-22

Registro 4:
Tipo: receita
Valor: 305.0
Data: 2023-11-22

Registro 5:
Tipo: despesa
Valor: -1699.0
Data: 2023-11-22

Registro 6:
Tipo: receita
Valor: 3683.0
Data: 2023-11-22

Registro 7:
Tipo: receita
Valor: 1401.0
Data: 2023-11-22

Registro 8:
Tipo: receita
Valor: 5395.0
Data: 2023-11-22

Registro 9:
Tipo: investimento
Valor: 8884.0
Data: 2023-11-22
Montante: 10580.99
Taxa de Juros: 6
Tempo de Aplic